In [1]:
import json
import pandas as pd
import os

In [4]:
def update_provincia_ids(json_path, csv_path, output_path):
    """
    Carga el GeoJSON de provincias y el CSV de códigos de provincia,
    y actualiza cada feature['properties']['id'] e 'id' con el código INE.
    """
    # 1) Leer GeoJSON
    with open(json_path, 'r', encoding='utf-8') as f:
        geo = json.load(f)

    # 2) Leer CSV de códigos
    df_codes = pd.read_csv(csv_path, dtype=str, delimiter=";")
    mapping = dict(zip(df_codes['name'], df_codes['cod_prov']))

    # 3) Actualizar cada feature
    missing = []
    for feat in geo.get('features', []):
        prov_name = feat['properties'].get('name')
        cod = mapping.get(prov_name)
        if cod:
            feat['properties']['id'] = cod
            feat['id'] = cod
        else:
            missing.append(prov_name)

    if missing:
        print(f"Advertencia: No se encontró código para: {sorted(set(missing))}")

    # 4) Guardar nuevo GeoJSON
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(geo, f, ensure_ascii=False, indent=2)

In [5]:
if __name__ == "__main__":
    json_input  = "../data/mapas/esp_prov.json"
    csv_input   = "../data/codigos_ine_prov_ccaa/codigos_prov_mapa.csv"
    json_output = "..data/mapas/esp_prov_codigos.json"
    update_provincia_ids(json_input, csv_input, json_output)
    print(f"GeoJSON actualizado guardado en: {json_output}")

GeoJSON actualizado guardado en: ..data/mapas/esp_prov_codigos.json


Ajustamos santa cruz y las palmas para que no queden tan alejadas del mapa

In [6]:
from shapely.geometry import shape, mapping
from shapely.affinity import translate, scale

def adjust_canarias(json_input, json_output, dx=10.0, dy=5.0, scale_factor=0.8):
    """
    Ajusta la posición y escala de las Islas Canarias en un GeoJSON de provincias.
    
    - dx, dy: traslado en grados (longitud, latitud).
    - scale_factor: factor de escala para reducir tamaño.
    """
    # Cargar GeoJSON
    with open(json_input, 'r', encoding='utf-8') as f:
        geo = json.load(f)

    for feat in geo['features']:
        prov_id = feat['properties'].get('id')
        # Provincias canarias: Las Palmas (35) y Santa Cruz de Tenerife (38)
        if prov_id in ('35', '38'):
            geom = shape(feat['geometry'])
            # Escalar centrado
            geom_scaled = scale(geom, xfact=scale_factor, yfact=scale_factor, origin='centroid')
            # Trasladar
            geom_moved = translate(geom_scaled, xoff=dx, yoff=dy)
            feat['geometry'] = mapping(geom_moved)

    # Guardar nuevo GeoJSON
    os.makedirs(os.path.dirname(json_output), exist_ok=True)
    with open(json_output, 'w', encoding='utf-8') as f:
        json.dump(geo, f, ensure_ascii=False, indent=2)

In [8]:
if __name__ == "__main__":
    input_path  = "..data/mapas/esp_prov_codigos.json"
    output_path = "../data/mapas/esp_prov_canarias_ajustada.json"
    # Ajusta con dx: +12 grados, dy: -8 grados, escala al 70%
    adjust_canarias(input_path, output_path, dx=12.0, dy=-8.0, scale_factor=0.7)
    print(f"GeoJSON con Canarias ajustadas guardado en: {output_path}")

GeoJSON con Canarias ajustadas guardado en: ../data/mapas/esp_prov_canarias_ajustada.json
